In [1]:
from ngsolve import *
from ngsolve import Draw, curl, grad
from netgen.occ import *
from netgen.gui import *
from coil_geometry import *

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2406
Using Lapack
Including sparse direct solver UMFPACK
Running parallel using 16 thread(s)


In [2]:
nwindings = 6  # Number of windings
wireradius = 0.001  # Radius of the wire
coilradius = 0.01  # Radius of the coil

# Geometrie laden
geo = create_homo_geometry(nwindings, wireradius, coilradius)

In [3]:
Draw(geo)

In [4]:
mesh = Mesh(
    geo.GenerateMesh()
)
mesh.Curve(1) 

 Face 1 / 10 (parameter space projection)
 Face 2 / 10 (parameter space projection)
 Face 3 / 10 (parameter space projection)
 Face 4 / 10 (parameter space projection)
 Face 5 / 10 (parameter space projection)
 Face 6 / 10 (parameter space projection)
 Face 7 / 10 (parameter space projection)
 Face 8 / 10 (parameter space projection)
 Face 9 / 10 (parameter space projection)
 Face 10 / 10 (parameter space projection)
 Delaunay meshing
 Remove Illegal Elements
 Delaunay meshing
 start tetmeshing
 Success !
 Volume Optimization
 Curve elements, order = 1


In [5]:
mesh.GetBoundaries()

('default',
 'default',
 'default',
 'default',
 'outer',
 'outer',
 'outer',
 'outer',
 'outer',
 'outer')

In [6]:
print("Materialien im Mesh:", mesh.GetMaterials())
print("Randbedingungen im Mesh:", mesh.GetBoundaries())

Materialien im Mesh: ('coil', 'air')
Randbedingungen im Mesh: ('default', 'default', 'default', 'default', 'outer', 'outer', 'outer', 'outer', 'outer', 'outer')


In [20]:
# Konstanten
mu = 4 * pi * 1e-7  # Magnetische Feldkonstante (S.2)
sigma_coil = 5.87e7  # Leitfähigkeit Kupfer (S.11)
sigma_air = 0 # Leitfähigkeit Luft
k = 1e-6  # Regularisierung

# Geometrische Parameter
i_c = 500 
A_c = pi * wireradius**2  
r = sqrt(x**2 + y**2 + z**2) + 1e-30

# Stromdichte j_c gemäß Aufgabenstellung
j_c = CoefficientFunction(
    ((nwindings * i_c / A_c) * (y / r), -(nwindings * i_c / A_c) * (x / r), 0)
)

In [23]:
# H(curl)-Raum definieren
order = 2
V = HCurl(mesh, order=order-1, nograds=True, dirichlet="outer")
u, v = V.TnT()
gfA = GridFunction(V)


# Bilinearform und Linearform
a = BilinearForm(V, symmetric=True)
a += (1/mu) * InnerProduct(curl(u), curl(v)) * dx
a += k * InnerProduct(u, v) * dx

# Preconditioner
pre = Preconditioner(a, "multigrid")
#pre = Preconditioner(a, "local")

# Linearform: Stromdichte
f = LinearForm(V)
f += InnerProduct(j_c, v) * dx("Coil")

# Assemble
with TaskManager():
    a.Assemble()
    pre.Update()  # <<-- Nicht vergessen!

    f.Assemble()

print(f"Matrix size: {a.mat.height} x {a.mat.width}")
print(f"Vector size: {f.vec.size}")
print(f"Präkonditionierer-Matrixgröße: {pre.mat.height} x {pre.mat.width}")

hcurl smoothingblocks, SmoothingType = 2
hcurl smoothingblocks, SmoothingType = 2
Matrix size: 48023 x 48023
Vector size: 48023
Präkonditionierer-Matrixgröße: 48023 x 48023


Thank you for using NGSolve


In [22]:
# Lösen
inv = CGSolver(a.mat, pre.mat, maxiter=200)
print(inv.GetSteps())

with TaskManager():
    gfA.vec.data = inv * f.vec

# Visualisierung
#Draw(curl(gfA), mesh, "MagneticField")
Draw(gfA, mesh, "VectorPotential", draw_surf=True, draw_vol=True)

170985786
0 nan


In [10]:
# Calculate the residual manually
residual_vec = f.vec - a.mat * gfA.vec
residual_norm = residual_vec.Norm()
 
# Check if the solver converged
steps = inv.GetSteps()
if steps < 200 and residual_norm < 1e-8:
    print(f"Solver converged in {steps} steps with residual norm {residual_norm}")
else:
    print(f"Solver did not converge: steps={steps}, residual norm={residual_norm}")
 
# Check GridFunction norm
solution_norm = gfA.vec.Norm()
if solution_norm > 0 and solution_norm < 1e10:  # Arbitrary upper bound for sanity check
    print(f"Solution norm is valid: {solution_norm}")
else:
    print("Solution norm is invalid or zero")
 
# Check if mesh materials and boundaries are correct
print("Materials in mesh:", mesh.GetMaterials())
print("Boundaries in mesh:", mesh.GetBoundaries())
 
# Debug matrix and vector sizes
print(f"Matrix size: {a.mat.height} x {a.mat.width}")
print(f"Vector size: {f.vec.size}")
 

Solver did not converge: steps=1, residual norm=23988.73528565578
Solution norm is invalid or zero
Materials in mesh: ('coil', 'air')
Boundaries in mesh: ('default', 'default', 'default', 'default', 'outer', 'outer', 'outer', 'outer', 'outer', 'outer')
Matrix size: 48023 x 48023
Vector size: 48023


In [11]:
B = curl(gfA)

In [12]:
import numpy as np

yiB = np.linspace(-.005, .005, 9)
XiB, YiB = np.meshgrid(yiB, yiB)
fieldlines = B._BuildFieldLines(
    mesh,
    np.array([XiB.flatten(), YiB.flatten(), np.zeros_like(XiB.flatten())]).T,
    num_fieldlines=10**3 // 5,
    randomized=False,
    length=1.,
    thickness=8e-4
)

Draw(B, mesh, objects=[fieldlines], draw_surf=False, min=0, max=0.1, settings={"Objects": {"Edges": False, "Surface": True, "Wireframe": False}})

lines = {
    "type": "lines",
    "position": np.array([[yi[k, :3], yi[k + 1, :3]] for k in range(yi.shape[0] - 1)]).flatten(),
    "name": "electron",
    "color": "blue"
}
lines2 = {
    "type": "lines",
    "position": np.array([[yi2[k, :3], yi2[k + 1, :3]] for k in range(yi2.shape[0] - 1)]).flatten(),
    "name": "electron2",
    "color": "orange"
}
lines3 = {
    "type": "lines",
    "position": np.array([[yi3[k, :3], yi3[k + 1, :3]] for k in range(yi3.shape[0] - 1)]).flatten(),
    "name": "electron3",
    "color": "green"
}

scene = Draw(
    B,
    mesh.Materials('coil'),
    draw_surf=False,
    objects=[lines, lines2, lines3],
    settings={"Objects": {"Edges": False, "Surface": True, "Wireframe": False}},
    clipping={"y": 1, "z": 0, "function": False, "grid_size": 100}
)

TypeError: Draw(): incompatible function arguments. The following argument types are supported:
    1. (cf: ngsolve.fem.CoefficientFunction, mesh: ngsolve.comp.Mesh, name: str, sd: int = 2, autoscale: bool = True, min: float = 0.0, max: float = 1.0, draw_vol: bool = True, draw_surf: bool = True, reset: bool = False, title: str = '', number_format: str = '%.3e', unit: str = '', **kwargs) -> None
    2. (gf: ngsolve.comp.GridFunction, sd: int = 2, autoscale: bool = True, min: float = 0.0, max: float = 1.0, **kwargs) -> None
    3. (mesh: ngsolve.comp.Mesh, **kwargs) -> None
    4. (arg0: object) -> None

Invoked with: <ngsolve.comp.GridFunctionCoefficientFunction object at 0x70fd10fcc3b0>, <ngsolve.comp.Mesh object at 0x70fd10fa5350>; kwargs: objects=[{'type': 'fieldlines', 'name': 'fieldlines', 'pstart': [], 'pend': [], 'value': [], 'thickness': 0.00013856406460551022}], draw_surf=False, min=0, max=0.1, settings={'Objects': {'Edges': False, 'Surface': True, 'Wireframe': False}}